In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
jobsUSData = pd.read_csv('Dataset For Job Recommendation System/jobsUsdata.csv')
jobsUSData.head(4)

,Unnamed: 0,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,0,1,1,Security Engineer/Technical Lead,Security Engineer/Technical Lead<p>Security Cl...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531.0,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,1,4,1,SAP Business Analyst / WM,SAP Business Analyst / WM<strong>NO Corp. to C...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217.0,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,2,7,1,P/T HUMAN RESOURCES ASSISTANT,P/T HUMAN RESOURCES ASSISTANT<b> <b> P/T HU...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792.0,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,3,8,1,Route Delivery Drivers,Route Delivery DriversCITY BEVERAGES Come to w...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [4]:
jobsUSbase = jobsUSData.iloc[0:20000]

In [5]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix = tf.fit_transform(jobsUSbase['Description'])
tfidf_matrix.shape

(20000, 883828)

In [6]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape
cosine_sim[0]
jobsUSbase.reset_index()

,index,Unnamed: 0,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,0,0,1,1,Security Engineer/Technical Lead,Security Engineer/Technical Lead<p>Security Cl...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531.0,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,1,1,4,1,SAP Business Analyst / WM,SAP Business Analyst / WM<strong>NO Corp. to C...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217.0,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,2,2,7,1,P/T HUMAN RESOURCES ASSISTANT,P/T HUMAN RESOURCES ASSISTANT<b> <b> P/T HU...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792.0,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,3,3,8,1,Route Delivery Drivers,Route Delivery DriversCITY BEVERAGES Come to w...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,4,4,9,1,Housekeeping,HousekeepingI make sure every part of their d...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,19999,78423,1,"Apartments.com Sales Rep, General Assignment O...","Apartments.com Sales Rep, General Assignment O...",NaN,Los Angeles,CA,US,90012.0,2012-03-15 15:20:47.997,2012-04-14 23:59:59
19996,19996,20000,78424,1,"Advertising Account Executive, Travel","Advertising Account Executive, TravelThe Adver...",NaN,Los Angeles,CA,US,90012.0,2012-03-15 18:20:57.653,2012-04-14 23:59:59
19997,19997,20001,78426,1,Inside Sales Representative/Account Manager,Inside Sales Representative/Account ManagerIf ...,NaN,Los Angeles,CA,US,90012.0,2012-03-29 14:03:26.35,2012-04-28 23:59:59
19998,19998,20002,78435,1,Support Services,"Support Services<P style=""MARGIN: 0in 0in 0pt""...","hardware, mainframe, platform, network, server",Austin,TX,US,78758.0,2012-03-12 18:10:02.447,2012-04-11 23:59:59


In [7]:
titles  = jobsUSbase['Title']
indices = pd.Series(jobsUSbase.index, index=jobsUSbase['Title'])


In [8]:
def get_recommnedation(title,cnt):
    similar_job_indices = []
    #def get_recommendation(title, no_of_jobs):
    i = indices[title]
    simulated_scores = enumerate(cosine_sim[i])
    simulated_scores = sorted(simulated_scores, key = lambda x: x[1],reverse = True)
    #print(simulated_scores)
    count = 0
    for i in simulated_scores:
        count = count+1
        similar_job_indices.append(i[0])
        if count == cnt:
            break
    print(similar_job_indices)
    for i in similar_job_indices:
        print(titles[i])  
    

In [9]:
get_recommnedation('SAP Business Analyst / WM',10)

[1, 14314, 6051, 18965, 19098, 5868, 19085, 5159, 5351, 14002]
SAP Business Analyst / WM
SAP WM Functional Consultant
SAP FI/CO Business Consultant
SAP Enterprise Architect
SAP HR Functional Consultant
SAP FI/CO Business Analyst
SAP Functional Analyst
SAP Basis Administrator
SAP Sales and Distribution Solution Architect
SAP BW Developer


In [10]:
get_recommnedation('Route Delivery Drivers',10)

[3, 4027, 48, 12696, 13481, 12689, 26, 7517, 31, 16340]
Route Delivery Drivers
Route Delivery Driver
TRACTOR TRAILER DRIVER
Route Delivery Driver - Lanham, MD
Class A CDL Truck Driver-Distribution / Route Delivery Driver
Los Angeles - Route Sales Representative
EXPERIENCED ROOFERS
Class A Route Drivers
DRIVERS
Route Supervisor


In [11]:
userData = pd.read_csv('Dataset For Job Recommendation System/users.tsv',delimiter = '\t')
userData.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [12]:
user_based_approach_US = userData.loc[userData['Country']=='US']
user_based_approach_US.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [13]:
user_based_approach_US = user_based_approach_US.iloc[0:15000]
user_based_approach_US

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15063,291341,1,Train,Haslett,MI,US,48840,Bachelor's,Business,NaN,4,23.0,Yes,Yes,5
15064,291345,1,Train,North Las Vegas,NV,US,89032,None,NaN,2005-06-01 00:00:00,5,6.0,No,No,0
15065,291392,1,Train,Mount Dora,FL,US,32757,High School,Not Applicable,1982-01-01 00:00:00,8,23.0,Yes,No,0
15066,291506,1,Train,Glendale,CA,US,91205,High School,Business Administration,2003-06-01 00:00:00,8,10.0,No,Yes,3


In [14]:
appsData = pd.read_csv('Dataset For Job Recommendation System/apps.tsv',delimiter = '\t')
appsData.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [15]:
user_based_approach_US['Major'] = user_based_approach_US['Major'].fillna('')
user_based_approach_US['DegreeType'] = user_based_approach_US['DegreeType'].fillna('')
user_based_approach_US['TotalYearsExperience'] = str(user_based_approach_US['TotalYearsExperience'].fillna(''))
user_based_approach_US['Details'] = user_based_approach_US['Major'] + user_based_approach_US['DegreeType'] + user_based_approach_US['TotalYearsExperience']

In [16]:
user_based_approach_US.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Details
0,47,1,Train,Paramount,CA,US,90723,High School,,1999-06-01 00:00:00,3,0 10\n1 8\n2 11\n3 ...,Yes,No,0,High School0 10\n1 8\n2 ...
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,0 10\n1 8\n2 11\n3 ...,Yes,No,0,AnthropologyMaster's0 10\n1 8\n...
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,0 10\n1 8\n2 11\n3 ...,Yes,Yes,5,Not ApplicableHigh School0 10\n1 ...
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,0 10\n1 8\n2 11\n3 ...,Yes,No,0,JournalismMaster's0 10\n1 8\n2 ...
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,0 10\n1 8\n2 11\n3 ...,Yes,No,0,Agricultural BusinessBachelor's0 10\n1 ...


In [17]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df =0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(user_based_approach_US['Details'])

In [18]:
tfidf_matrix.shape

(15000, 8299)

In [19]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [20]:
cosine_sim[0]

array([1.        , 0.24082208, 0.62216091, ..., 0.62216091, 0.2858051 ,
       0.35454357])

In [21]:
user_based_approach_US = user_based_approach_US.reset_index()
userid = user_based_approach_US['UserID']
indices = pd.Series(user_based_approach_US.index, index=user_based_approach_US['UserID'])

In [22]:
def get_recommendations_userwise(userid,cnt):
    user_indices = []
    idx = indices[userid]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    count = 0
    for i in sim_scores:
        count = count+1
        user_indices.append(i[0])
        if count == cnt:
            break
    return user_indices

In [23]:
print("---- Top N similar users with userId: 123 ---")
get_recommendations_userwise(123,10)

---- Top N similar users with userId: 123 ---


[4, 165, 246, 1152, 1162, 1460, 1617, 1815, 2128, 2173]

In [24]:
def get_job_id(userid_list):
    jobs_userwise = appsData['UserID'].isin(userid_list)
    df1 = pd.DataFrame(data = appsData[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobsUSData['JobID'].isin(joblist)
    df_temp = pd.DataFrame(data = jobsUSData[Job_list], columns = ['JobID','Title','Description','State','City'])
    return df_temp
    

In [25]:
get_recommendations_userwise(123,10)

[4, 165, 246, 1152, 1162, 1460, 1617, 1815, 2128, 2173]

In [26]:
get_job_id(get_recommendations_userwise(123,10))

,JobID,Title,Description,State,City
914833,522863,Customer Service Representatives Needed ASAP!,Customer Service Representatives Needed ASAP!&...,MD,Timonium
968218,1037808,Customer Service Representative,Customer Service RepresentativeAre you looking...,MD,Frederick
972747,1083927,CAD Designer,"CAD Designer<p style=""text-align: justify""><b>...",MD,Towson
974638,1101366,Catering Office Coordinator,Catering Office CoordinatorPrestigious private...,MD,Baltimore
1005881,285277,Weight Loss Counselor,"Weight Loss Counselor<P style=""TEXT-ALIGN: cen...",MN,Minneapolis


In [27]:
get_recommendations_userwise(80,5)

[2, 7, 9, 15, 18]

In [28]:
get_job_id(get_recommendations_userwise(80,5))

,JobID,Title,Description,State,City
478628,309823,Teller,TellerOur vision is to satisfy all our custome...,VA,Roanoke
527428,703889,"Customer Service Representative, Now Accepting...","Customer Service Representative, Now Accepting...",VA,Roanoke
596359,136489,UPS Part Time Package Handler,UPS Part Time Package Handler<hr>\r<strong>\r<...,TX,Houston
657023,617374,Full Time Fork Lift Driver,Full Time Fork Lift Driver<p>The Dump is curr...,TX,Houston
680692,809208,Now Hiring All Trades,Now Hiring All TradesNow hiring all trades ASA...,TX,Houston
880059,187311,Helper,Helper<p>We are looking for:<br />\r2 helpers ...,TX,Houston
942389,787477,Blasters,Blasters<p>We are looking for: <br />\r2 exper...,TX,Houston
946334,828603,General Labor,General Labor<b>General Labor</b>\r<br />\r<br...,TX,Houston
